<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Определение токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Требуемое значение метрики качества *F1* - не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем библиотеки.

In [13]:
import pandas as pd

import numpy as np

import re

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier


import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Загрузим и прочитаем данные.

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


В нашем распоряжении датафрейм из двух столбцов и 159292 строк. В столбце ```text``` содержатся комментарии, а в столбце ```toxic``` метка - 1, если комментарий токсичного содержания и 0 в противном случае. Проверим, есть ли пропуски и дубликаты.

In [3]:
def check_data(data):
    # Проверка на пропуски
    missing_data = data.isnull().sum()
    
    # Проверка на дубликаты
    duplicate_count = data.duplicated().sum()
    
    return missing_data, duplicate_count

missing_data, duplicate_count = check_data(data)
print("Пропуски в данных:")
print(missing_data)
print("\nКоличество дубликатов:", duplicate_count)


Пропуски в данных:
text     0
toxic    0
dtype: int64

Количество дубликатов: 0


Пропусков в данных не обнаружено, дубликаты также отсутствуют. Можно переходить к предобработке данных.

In [4]:
def preprocess_text(text):
    # Заменяем символы переноса строки пробелами
    text = text.replace('\n', ' ')
    
    # Удаляем все символы, кроме букв и пробелов
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Удаляем все лишние пробелы (где их по несколько штук подряд)
    text = re.sub(r'\s+', ' ', text)
    
    # Приводим текст к нижнему регистру
    text = text.lower()
    
    return text

# Применяем функцию
data['text'] = data['text'].apply(preprocess_text)

# Смотрим, что получилось
data

,text,toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
3,more i cant make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0
...,...,...
159446,and for the second time of asking when your vi...,0
159447,you should be ashamed of yourself that is a ho...,0
159448,spitzer umm theres no actual article for prost...,0
159449,and it looks like it was actually you who put ...,0


Лишние символы исчезли, текст перешёл в нижний регистр.

In [5]:
# Создаем список стоп-слов
stop_words = set(stopwords.words("english"))

# Создаем функцию для лемматизации текста с учетом частей речи
def lemmatize_text_with_pos(text):
    def get_wordnet_pos(tag):
        tag = tag[0].upper()
        tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag([word])[0][1])) for word in words]
    return ' '.join(lemmatized_words)

# Применяем функцию лемматизации к столбцу с текстами
data['text'] = data['text'].apply(lemmatize_text_with_pos)

# Смотрим, что получилось
data

,text,toxic
0,explanation why the edits make under my userna...,0
1,daww he match this background colour im seemin...,0
2,hey man im really not try to edit war it just ...,0
3,more i cant make any real suggestion on improv...,0
4,you sir be my hero any chance you remember wha...,0
...,...,...
159446,and for the second time of ask when your view ...,0
159447,you should be ashamed of yourself that be a ho...,0
159448,spitzer umm there no actual article for prosti...,0
159449,and it look like it be actually you who put on...,0


Лемматизация проведена.

**Вывод:** Мы успешно провели предобработку текстов, включая удаление символов новой строки, удаление лишних символов, удаление лишних пробелов, приведение к нижнему регистру и лемматизацию текста с учетом частей речи. Теперь данные готовы к использованию для обучения модели классификации. 

## Обучение

Проверим баланс классов.

In [6]:
class_balance = data['toxic'].value_counts()
print(class_balance)

0    143106
1     16186
Name: toxic, dtype: int64


У нас имеет место дисбаланс классов, где класс 0 (нет токсичности) имеет намного больше экземпляров, чем класс 1 (токсичные комментарии). Для решения этой проблемы с дисбалансом классов можно использовать различные стратегии. Одной из них является взвешивание классов при обучении модели.

Воспользуемся параметром ```class_weight``` в моделях машинного обучения. Он позволяет установить веса классов, которые учитывают дисбаланс. Веса будут выставлены так, чтобы класс 1 (токсичные комментарии) имел больший вес, чем класс 0.

Разделим данные на обучающую, валидационную и тестовую выборки.

In [8]:
# Разделяем данные на обучающую и тестовую выборки
features = data['text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=1337, stratify=target)

Приступим к обучению моделей.

In [11]:
%%time
# Логистическая регрессия
lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('model', LogisticRegression(class_weight='balanced'))
])

lr_params = {
    'model__solver': ['liblinear'],
    'model__C': [0.1, 1, 10],
    'model__max_iter': [250]
}

lr_grid = GridSearchCV(lr_pipeline, lr_params, cv=3, scoring='f1', n_jobs=-1)
lr_grid.fit(features_train, target_train)

# Decision Tree
dt_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('model', DecisionTreeClassifier(class_weight='balanced'))
])

dt_params = {
    'model__max_depth': [None, 2, 12, 24],
    'model__min_samples_split': [2, 5, 10]
}

dt_grid = GridSearchCV(dt_pipeline, dt_params, cv=3, scoring='f1', n_jobs=-1)
dt_grid.fit(features_train, target_train)

# Random Forest
rf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('model', RandomForestClassifier(class_weight='balanced'))
])

rf_params = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 10, 20, 30]
}

rf_grid = GridSearchCV(rf_pipeline, rf_params, cv=3, scoring='f1', n_jobs=-1)
rf_grid.fit(features_train, target_train)

# Получение лучших параметров и оценки F1 для каждой модели
best_lr_params = lr_grid.best_params_
best_lr_f1_score = lr_grid.best_score_

best_dt_params = dt_grid.best_params_
best_dt_f1_score = dt_grid.best_score_

best_rf_params = rf_grid.best_params_
best_rf_f1_score = rf_grid.best_score_

print("Лучшие параметры и оценки F1 для логистической регрессии:")
print(best_lr_params)
print("Лучший F1 Score для логистической регрессии:", best_lr_f1_score)

print("Лучшие параметры и оценки F1 для Decision Tree:")
print(best_dt_params)
print("Лучший F1 Score для Decision Tree:", best_dt_f1_score)

print("Лучшие параметры и оценки F1 для Random Forest:")
print(best_rf_params)
print("Лучший F1 Score для Random Forest:", best_rf_f1_score)

# Находим модель с наилучшими параметрами
best_model = None
best_model_name = None
best_params = None
best_f1_score = -1

for model_name, grid_result in [('Logistic Regression', lr_grid), ('Decision Tree', dt_grid), ('Random Forest', rf_grid)]:
    if grid_result.best_score_ > best_f1_score:
        best_model = grid_result.best_estimator_
        best_model_name = model_name
        best_params = grid_result.best_params_
        best_f1_score = grid_result.best_score_

# Выводим имя лучшей модели и её гиперпараметры
print("Лучшая модель:", best_model_name)
print("Лучшие гиперпараметры:", best_params)

Лучшие параметры и оценки F1 для логистической регрессии:
{'model__C': 10, 'model__max_iter': 250, 'model__solver': 'liblinear'}
Лучший F1 Score для логистической регрессии: 0.7561318165467771
Лучшие параметры и оценки F1 для Decision Tree:
{'model__max_depth': None, 'model__min_samples_split': 2}
Лучший F1 Score для Decision Tree: 0.6032850107846731
Лучшие параметры и оценки F1 для Random Forest:
{'model__max_depth': None, 'model__n_estimators': 200}
Лучший F1 Score для Random Forest: 0.5923569397364705
Лучшая модель: Logistic Regression
Лучшие гиперпараметры: {'model__C': 10, 'model__max_iter': 250, 'model__solver': 'liblinear'}
CPU times: user 2h 3min 24s, sys: 1min 30s, total: 2h 4min 55s
Wall time: 2h 5min 1s


Из трёх моделей только Логистическая Регрессия достигла необходимого результата по f1-мере (0.75). Её f1 составила 0.756. Теперь модель нужно проверить на тестовой выборке, и в случае, если результат не будет хуже, чем 0.75, можно считать эту модель подходящей для наших задач.

In [12]:
# Проверка лучшей модели на тестовой выборке
test_predictions = best_model.predict(features_test)
test_f1_score = f1_score(target_test, test_predictions)

print("F1 Score на тестовой выборке:", test_f1_score)

F1 Score на тестовой выборке: 0.7599942685198452


Значение f1 на тестовой выборке составило 0.76, что позволяет сделать вывод, что данная модель полностью отвечает нашим требованиям.

## Выводы

В рамках данного проекта были выполнены следующие этапы:

1. Загрузка данных: Начальным шагом было считывание данных из файла 'toxic_comments.csv' с помощью библиотеки Pandas. Выведена общая информация о данных для оценки объема и качества данных.

2. Предобработка данных: Проведена предобработка текстовых данных, включая удаление символов новой строки, удаление лишних символов, приведение текста к нижнему регистру, и лемматизация текста с учетом частей речи. Также создан список стоп-слов для английского языка.

3. Балансировка классов: Обнаружен дисбаланс классов в целевой переменной и применена балансировка классов для моделей.

4. Разделение данных: Данные были разделены на обучающую и тестовую выборки. При этом был соблюдён одинаковый баланс классов в обоих выборках.

5. Обучение моделей: Обучены три различные модели - логистическая регрессия, дерево решений и случайный лес, с использованием метода кросс-валидации и GridSearchCV для подбора оптимальных гиперпараметров. Для каждой модели подобраны наилучшие параметры и вычислен F1 Score на обучающей выборке.

6. Тестирование модели: Лучшая модель, в данном случае логистическая регрессия, была протестирована на тестовой выборке. Полученный F1 Score на тестовых данных составил 0.760, что удовлетворяет требованиям проекта (F1 Score не менее 0.75).

Таким образом, проект успешно выполнен, и модель готова к использованию для классификации токсичных комментариев в интернет-магазине "Викишоп".